[View in Colaboratory](https://colab.research.google.com/github/SendilCannane/Neural-Network/blob/master/SimpleNN.ipynb)

In [81]:
#%matplotlib.inline

from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from sklearn.metrics import confusion_matrix
import IPython
print(IPython.sys_info())

{'commit_hash': 'b467d487e',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/usr/local/lib/python3.6/dist-packages/IPython',
 'ipython_version': '5.5.0',
 'os_name': 'posix',
 'platform': 'Linux-4.14.65+-x86_64-with-Ubuntu-18.04-bionic',
 'sys_executable': '/usr/bin/python3',
 'sys_platform': 'linux',
 'sys_version': '3.6.6 (default, Sep 12 2018, 18:26:19) \n'
                '[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'}


In [16]:
from tensorflow.examples.tutorials.mnist import input_data
data_mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
data_train = data_mnist.train
data_test = data_mnist.test

In [76]:
data_test.cls = np.argmax(data_test.labels, axis=1)
data_test.cls[0:5]



array([7, 2, 1, 0, 4])

In [20]:
data_train.images.shape

(55000, 784)

In [21]:
data_train.labels.shape

(55000, 10)

In [22]:
data_test.images.shape

(10000, 784)

In [23]:
data_test.labels.shape

(10000, 10)

In [0]:
learning_rate = 0.2
num_steps = 500
batch_size = 128
display_step = 100

In [0]:
num_h1_nodes = 256
num_h2_nodes = 256
num_inputs = 784 #data_test.images.shape (10000, 784)
num_classes = 10 #data_test.labels.shape (10000, 10)

In [0]:
x=tf.placeholder(tf.float32, [None,num_inputs])

In [0]:
Y=tf.placeholder(tf.float32,[None,num_classes])

In [0]:
#weights and biases

weights = {
    'h1w' : tf.Variable(tf.random_normal([num_inputs, num_h1_nodes])),
    'h2w' : tf.Variable(tf.random_normal([num_h1_nodes, num_h2_nodes])),
    'out' : tf.Variable(tf.random_normal([num_h2_nodes, num_classes]))
    
}

biases = {
    'h1b1': tf.Variable(tf.random_normal([num_h1_nodes])),
    'h2b2': tf.Variable(tf.random_normal([num_h2_nodes])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
#Model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1w']), biases['h1b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2w']), biases['h2b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [0]:
logits = neural_net(x)

# Define loss and optimizer
# In order to use the cross-entropy to guide the optimization of the model's variables 
# we need a single scalar value, so we simply take the average of the cross-entropy for all the image classifications.
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
# We need a few more performance measures to display the progress to the user.
# This is a vector of booleans whether the predicted class equals the true class of each image.
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))


#This calculates the classification accuracy by first type-casting the vector of 
#booleans to floats, so that False becomes 0 and True becomes 1, and then calculating the average of these numbers
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [0]:
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)

In [89]:
# Start training with train data
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
      #Split the data to be trained as batch_x and batch_y
        batch_x, batch_y = data_train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={x: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Training Finished!")
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: data_test.images,
                                      Y: data_test.labels}))
    
    # Get the true classifications for the test-set.
    cls_true = data_test.cls
    #data_test.cls = np.argmax(data_test.labels, axis=1)
    # Get the predicted classifications for the test-set.
    cls_pred = sess.run(y_pred_cls, feed_dict={x: data_test.images,Y: data_test.labels})
    #cls_pred = sess.run(test_pred, feed_dict={x: data_test.images,
     #                                 Y: data_test.labels})
      
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    # Print the confusion matrix as text.
    print ("***********Confusion matrix************")
    print(cm)

Step 1, Minibatch Loss= 8031.0151, Training Accuracy= 0.359
Step 100, Minibatch Loss= 330.7074, Training Accuracy= 0.820
Step 200, Minibatch Loss= 222.9307, Training Accuracy= 0.859
Step 300, Minibatch Loss= 60.9252, Training Accuracy= 0.867
Step 400, Minibatch Loss= 102.4662, Training Accuracy= 0.883
Step 500, Minibatch Loss= 33.4525, Training Accuracy= 0.914
Training Finished!
Testing Accuracy: 0.8637
***********Confusion matrix************
[[ 916    0    4    1    3    5   18   16   16    1]
 [   0 1098    6    1    0    1    4    2   23    0]
 [   9   17  890   39   17    3   17    8   30    2]
 [   6    5   20  811    0  115    3    9   37    4]
 [   0    9    6    6  916    5   12    4   14   10]
 [  15    5    6   29    8  714   18   12   81    4]
 [  19    9    9    0   10   13  876    0   22    0]
 [   3   17   26   27    7    3    0  918    2   25]
 [   3   36    8   45   10   17    4   25  821    5]
 [   6   11    2   18  184   21    1   70   19  677]]
